# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
import winsound

In [2]:
%%time
data = get_date_list(data)
data = read_data(data)

Read dataframe with 16797607 records.
Dataframe has 240300 records between (datetime.datetime(2021, 1, 5, 0, 0), datetime.datetime(2021, 1, 6, 0, 0)).
Record num : 240300
Wall time: 18 s


In [3]:
for data['i'] in tqdm(range(0, data['df_len'])):
    
    data['df']['DateTime_frmt'][data['i']] = dt.datetime.strptime(data['df']['DateTime'][data['i']].split(".")[0],"%Y%m%d %H:%M:%S")
    
    data['ask'] = data['df']['Ask'][data['i']]
    data['bid'] = data['df']['Bid'][data['i']]  
    
    data['tick'] = (data['df']['Ask'][data['i']] + data['df']['Bid'][data['i']])/2
    data['df']['tick'][data['i']] = data['tick']
    
        
    if 'sema' in data['included_loops']:  
        if len(data['sema_list']) < data['sema_len']:
            data =  before_sema(data)
            continue
        if len(data['sema_list']) == data['sema_len']:
            data = after_sema(data)                 

    if 'slema' in data['included_loops']:
        if len(data['slema_list']) < data['slema_len']:
            data =  before_slema(data)
            continue
        if len(data['slema_list']) == data['slema_len']:
            data = after_slema(data)                 
    
    if 'lema' in data['included_loops']:    
        if len(data['lema_list']) < data['lema_len']:
            data =  before_lema(data)
            continue
        if len(data['lema_list']) == data['lema_len']:
            data = after_lema(data)                 
    
    data = get_ohlc(data)    
    data = get_trend_fwd(data)    
    data = get_ohlc_small(data)    
    data = get_trend_fwd_small(data)    
    
    
#     data = get_position(data)
    
#     if not data['position']:
#         continue
        
#     # Get Dirs --------------------------------
#     if len(data['dir_list']) < 2:
#         data['dir_list'].append(data['position'])   
#         continue

#     elif len(data['dir_list']) == 2:
#         data = get_cross_dir(data)
#     # ----------------------------------------------------------  
    
#     data = calculate_pl(data)
#     data = simple_take_profit(data)
#     data = tick_close(data)
#     data = simple_stop_loss(data)
#     data = make_order(data)

100%|█████████████████████████████████| 240300/240300 [39:50<00:00, 100.51it/s]


In [4]:
num_rec = len(data["df"])
hours = np.round((list(data['df']['DateTime_frmt'])[-1] - list(data['df']['DateTime_frmt'])[0]).total_seconds() / 60 / 60 , 2)
recsphour = int(num_rec/hours)

print(f'Records   : {num_rec}')
print(f'Hours     : {hours}')
print(f'Recs/hour : {recsphour}')
print(f'Recs/min  : {int(recsphour/60)}')
print(f'Recs/sec  : {np.round(recsphour/3600, 1)}')

data['df'].head()

Records   : 240300
Hours     : 48.0
Recs/hour : 5006
Recs/min  : 83
Recs/sec  : 1.4


,DateTime,Bid,Ask,tick,DateTime_frmt,sema,slema,lema,position,to_order,...,pl_move_min,h,l,small_h,small_l,long_open,long_close,short_open,short_close,pl
0,20210105 00:00:00.070,1.22525,1.22531,1.225280,2021-01-05 00:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.22528,NaN,NaN,NaN,NaN,NaN,NaN
1,20210105 00:00:02.487,1.22525,1.22530,1.225275,2021-01-05 00:00:02,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20210105 00:00:03.611,1.22526,1.22528,1.225270,2021-01-05 00:00:03,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20210105 00:00:03.662,1.22522,1.22527,1.225245,2021-01-05 00:00:03,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20210105 00:00:03.764,1.22521,1.22526,1.225235,2021-01-05 00:00:03,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
plot_graph(data)

In [6]:
data['df'].to_csv('data/temp.csv')

In [7]:
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)
send_telegram_message(f'Run Complete')

winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)
send_telegram_message(f'------------\n{sum(pl_list)}\n------------\n{pl_list}\n------------')